# Code examples for working with Azure Open AI

## Prerequisites
The examples below use Python 3. Environment variables are stored in a `.env` file and loaded using `dotenv` package. You can load them with your method of choice.

Here is an example of a `.env` file. (Note: sometimes file starting with `.` are hidden and can't be seen by your OS file browser.)

In [ ]:
%%writefile .env
OPENAI_ENDPOINT=https://<your deployment name>.openai.azure.com/
OPENAI_API_KEY=<your key>
OPENAI_DEPLOYMENTID=<name (id) of your OpenAI model deployment for completion>
OPENAI_EMBEDDING_DEPLOYMENTID=<name (id) of your OpenAI model deployment for word embedding>
# below variables are needed if you'd like to run cognitive svc translator
# create a translator resource under cognitive services
AZURE_COGNITIVE_SVC_TEXT_ENDPOINT=https://api.cognitive.microsofttranslator.com/ 
AZURE_COGNITIVE_SVC_DOC_ENDPOINT=https://<your deployment name>.cognitiveservices.azure.com/
AZURE_COGNITIVE_SVC_API_KEY=<your cog svc key>
AZURE_COGNITIVE_SVC_REGION=<your resource region>

The following packages were installed to run the sample codes.
- `dotenv` to load `.env` file
- `openai` for python SDK examples

## Example 1: Sentiment analysis (Rest API) 


In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import requests
import json
import os 


openai_endpoint = os.getenv('OPENAI_ENDPOINT')
openai_key = os.getenv('OPENAI_API_KEY')
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
api_version = "2022-12-01"

fastfood_reviews = [
    "I went in with a large order. Maurice was cool, and made sure that everything was perfect.  This was my first time at this location, and I really don't expect good customer service from McDonald's.  Glad to see that this location is better then most.",
    "It took us 35 minutes to get through the drive thru. There were a lot of cars behind us and in front of us. The staff inside were just nonchalant walking around like it's a easy simple day. Then they got our order wrong. I wasn't about to wait another 35 minutes.",
    "Very slow line, they must have been short staffed. And the electronics went down so I could not order with points or coupons online in advance. Big Macs are always good though. The fries were very good.",
]

question = 'what is the sentiment of each of the reviews: Give the response in one line'

prompt = question+'\n'
for i,review in enumerate(fastfood_reviews):
    prompt += f'{i+1}. {review}\n'


payload = {
    # The prompt(s) to generate completions for, encoded as a string, a list of strings, or a list of token lists. Note that <\|endoftext\|> is the document separator that the model sees during training, so if a prompt isn't specified the model will generate as if from the beginning of a new document.
    "prompt":prompt,
    
    # The maximum number of tokens to generate in the completion. The token count of your prompt plus max_tokens can't exceed the model's context length. Most models have a context length of 2048 tokens (except davinci-codex, which supports 4096).
    "max_tokens": 1000,
    
    # What sampling temperature to use. Higher values means the model will take more risks. Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer. We generally recommend altering this or top_p but not both.
    "temperature":0.5,
    
    # for a list of all available parameters, visit:
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#completions
}

response = requests.post(
    f"{openai_endpoint}/openai/deployments/{deployment_id}/completions?api-version={api_version}", 
    json.dumps(payload), 
    headers={"api-key": openai_key}
).json()

print(f'''Here is the response of the above code: {response['choices'][0]['text']}''')


## Example 2: Text summarization (Python SDK - Completion)

In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
openai.api_version = "2022-12-01"

prompt = "Summerize the following document:\n"
# sample.txt contains 3 paragraphs about Giraffes from wikipedia
with open('sample.txt') as f:
  prompt += f.read()

# print(prompt)

response = openai.Completion.create(
  engine=deployment_id,
  prompt=prompt,
  temperature=0.7,
  max_tokens=200,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(f''' Here is the result of the above code: {response.choices[0].text}''')

## Example 3: Word embedding (Python SDK - Embedding)


In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")
deployment_id = os.getenv('OPENAI_EMBEDDING_DEPLOYMENTID')
openai.api_version = "2022-12-01"

def getEmbedding(input: str):
    response = openai.Embedding.create(
        engine=deployment_id,
        input=input,
    )
    return response.data[0].embedding


inputs = [
    "coffee", "tea", "sugar","milk","breakfast","cream", "chamomile","green tea",
    "laptop", "cpu", "monitor", "memory","storage", "keyboard","mouse", "headphone",
]
embeddings = [getEmbedding(x) for x in inputs]



In [ ]:
%matplotlib inline
import numpy as np 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

embeddings_vec = np.array(embeddings)

reduced_vec = PCA(random_state=0).fit_transform(embeddings_vec)[:,:3]

all = [reduced_vec[:8],
       reduced_vec[8:16],
]

fig = plt.figure()
ax = fig.add_subplot()
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
# for p in reduced_vec :
for v in all:
    xs = [p[0] for p in v]
    ys = [p[1] for p in v]
    zs = [p[2] for p in v]
    ax.scatter(xs, ys, marker='o')

print("Below is the resulting plot of the above code: ")
plt.show()


## Example 4: Few-shot learning (Rest API)  

In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import requests
import json
import os 

openai_endpoint = os.getenv('OPENAI_ENDPOINT')
openai_key = os.getenv('OPENAI_API_KEY')
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
api_version = "2022-12-01"


prompt = '''Q: how do you draw a car with shapes?
'''

payload = {"prompt":prompt,"max_tokens": 1000,"temperature":0.5,}

response = requests.post(
    f"{openai_endpoint}/openai/deployments/{deployment_id}/completions?api-version={api_version}", 
    json.dumps(payload), 
    headers={"api-key": openai_key}
).json()

print(f'''{response['choices'][0]['text']}''')

In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import requests
import json
import os 

openai_endpoint = os.getenv('OPENAI_ENDPOINT')
openai_key = os.getenv('OPENAI_API_KEY')
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
api_version = "2022-12-01"


prompt = '''
Q: how do you draw a house with shapes?
A: by drawing a triabgle on top of a square.

Q: how do you draw a star with shapes?
A: by drawing 5 triangles on edges of a pentagon.

Q: how do you draw a car with shapes?
'''

payload = {"prompt":prompt,"max_tokens": 1000,"temperature":0.5,}

response = requests.post(
    f"{openai_endpoint}/openai/deployments/{deployment_id}/completions?api-version={api_version}", 
    json.dumps(payload), 
    headers={"api-key": openai_key}
).json()

print(response['choices'][0]['text'])

## Pricing
Azure Open AI usage is charged based on token consumption. You will be charged for both tokens sent and received.
The following table is current as of February 2023.

|Series      | Model | Inferencing cost per 1000 tokens
|------------|------|---------
|Base        |Ada    | $0.0004
|Base        |Babbage| $0.0005
|Base        |Curie  | $0.002
|Base        |Davinci| $0.02
|Codex       |Code-Cushman | $0.024
|Codex       |Code-Davinci | $0.10
|Embedding   | Ada   | $0.004
|Embedding   |Babbage| $0.005
|Embedding   |Curie  | $0.02
|Embedding   |Davinci| $0.20


## Example 5: Cost estimation (Python SDK - Completion)


In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
openai.api_version = "2022-12-01"

unit_cost={
  "completion":{"ada":0.0004, "babbage":0.0005, "curie":0.002, "davinci":0.02},
  "embedding" :{"ada":0.0040, "babbage":0.0050, "curie":0.020, "davinci":0.20}
}

prompt = "Summerize the following article:\n"
with open('few-shot.txt') as f:
  prompt += f.read()

# print(prompt)

response = openai.Completion.create(
  engine=deployment_id,
  prompt=prompt,
  temperature=0.7,
  max_tokens=2000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

token_cost = unit_cost['completion']['davinci']

print(f"""Response: 
{response['choices'][0]['text']}
--------------------------
Usage:
  - Prompt tokens:     {response['usage']['prompt_tokens']}
  - Completion tokens: {response['usage']['completion_tokens']}
  - Total tokens:      {response['usage']['total_tokens']}
Estimated cost:
  {response['usage']['total_tokens']} tok x {token_cost} ($/1000 tok) = ${response['usage']['total_tokens']*token_cost*0.0001:.4f}
""")

## Example 6: Translation 
### 6.1. OpenAI Python SDK

In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import openai
import os
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")
deployment_id = os.getenv('OPENAI_DEPLOYMENTID')
openai.api_version = "2022-12-01"

prompt = "Translate this to Farsi:\n I would really like to drive your car around the block a few times!"


# print(prompt)

response = openai.Completion.create(
  engine=deployment_id,
  prompt=prompt,
  temperature=0.7,
  max_tokens=2000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response.choices[0].text)

### 6.2. Cognitive Services - Translator (Rest API)

In [ ]:
# load environment variables contaning endpoint and API key
from dotenv import load_dotenv
load_dotenv()

import os, uuid, requests
endpoint = os.getenv('AZURE_COGNITIVE_SVC_TEXT_ENDPOINT')
api_key = os.getenv('AZURE_COGNITIVE_SVC_API_KEY')
location = os.getenv('AZURE_COGNITIVE_SVC_REGION')

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['fa']
}

headers = {
    'Ocp-Apim-Subscription-Key': api_key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': 'I would really like to drive your car around the block a few times!'
}]

request = requests.post(f'{endpoint}translate', params=params, headers=headers, json=body)
response = request.json()

print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))